In [1]:
import requests
import ast
import pandas as pd
import numpy as np
import sys
sys.path.append("../")
from scr.driver import driver
from scr.infoproducts_pull import get_id_info
from scr.stores import get_stores_pull
from time import sleep

In [4]:
product= 4470305300

store = {
    "physicalStoreId":13636
}


stock = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
res = requests.get(stock.format(f"0{product}"), params=store, headers=headers).json()

In [5]:
res

{'stocks': [{'datatype': 'stock',
   'physicalStoreId': 13636,
   'sizeStocks': [{'datatype': 'sizeStock',
     'sizeId': 2,
     'quantity': 0,
     'size': 102},
    {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 1, 'size': 103},
    {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}

In [6]:
def get_stock_pull(search_terms,lat,lng):
        

        products = get_id_info(search_terms)

        

        store_info = get_stores_pull(lat, lng)
        store_ids = [element["id"] for element in store_info]

        stock_url = "https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/{}"
        headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
        stock_params = {"physicalStoreId": store_ids}

        urls = [stock_url.format("".join(p["id_prod"]).rjust(11,"0")) for p in products]
        print(urls)
        
        for i,product in enumerate(products):
            headers ={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 11_2_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36"}
            products[i]["stock"] = requests.get(urls[i],params=stock_params,headers=headers).json()
        
        return products

In [18]:
prod = get_stock_pull("chaqueta amarilla",40.416050951604056, -3.7072654155715603)

Cazadora confeccionada en 100% algodón con detalle de bolsillos delanteros de tapeta, cuello clásico y manga larga.
Chaleco de ochos corto con cuello de pico y detalle de rib en el bajo.
['https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/05713324309', 'https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/04574303300']


In [19]:
prod

[{'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502620977.html?colorId=309',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/5713/324/309/5713324309_2_1_2.jpg?t=1619715493366',
  'product_name': 'Cazadora algodón bolsillos delanteros',
  'price': '25,99 € ',
  'id_prod': ['5713324309'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 4,
       'quantity': 1,
       'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 1,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 1, 'size': 104}]}]}},
 {'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502660734.html?colorId=300',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/4574/303/300/4574303300_2_1_2.jpg?t=1613464621559',
  'product_name': 'Chaleco ochos corto',
  'pric

In [7]:
def sacar_cosas(diccionario):
    
    url  = diccionario.get('url')
    image  = diccionario.get('image')
    product_name  = diccionario.get('product_name')
    price  = diccionario.get('price')
    
    items = {"Product_name": [], "url": [], "Image":[], "Price":[], "Quantity":[], "Size":[], "Tienda": []}
    
    size_ = {101:"XS", 102:"S", 103:"M", 104:"L", 105:"XL", 106:"XXL",125:"S-M",131:"L-XL"}
    items["url"].append(url)
    
    for k,v in diccionario["stock"].items():
        print((v[0]))
        tienda = v[0].get('physicalStoreId')
    for i in v[0]["sizeStocks"]:
        #size = i["sizeStocks"].get('sizeId')
        print(i)
        size  = i.get('size')
        converted_size = size_[size]
        quantity  = i.get('quantity')
        
        if quantity <= 0:
            quantity = "Lo siento, no tenemos existencias!☹️"
        elif quantity == 1:
            quantity = "Últimas existencias 👀"
                
        elif quantity >= 2:
            quantity = "El producto está en tienda!🙂"
                
        elif quantity >= 4:
            quantity = "Suficientes existencias!!😁"
        
        
        items["Quantity"].append(quantity)
        items["Size"].append(converted_size)
        items["url"].append(url)
        items["Image"].append(image)
        items["Product_name"].append(product_name)
        items["Price"].append(price)
        items["Tienda"].append(tienda)
        
        
                
    return items

In [10]:
def productos_finales(search_items, lat, lon):
    prod = get_stock_pull(search_items,lat,lon)
    i_busqueda = list(map(sacar_cosas, prod))
    df = pd.DataFrame(i_busqueda)
    df2 = df.explode("Size")
    df3 = df2.explode("Quantity")
    df4 = df3.drop_duplicates(subset=["Quantity", "Size"])
    return df4 

In [9]:
def limpieza_df():
    df2 = df.explode("Size")
    df3 = df2.explode("Quantity")
    df4 = df3.drop_duplicates(subset=["Quantity", "Size"])
    return df4

In [11]:
productos_finales("blusa amarilla", 40.416050951604056, -3.7072654155715603)

Sobrecamisa oversize de cuadros amarillos con bolsillos delanteros, cierre de botones y de manga larga.
Camisa tie-dye con detalles de manchas en tonos amarillos y de manga corta, con cierre de botones delanteros y en viscosa.
Chaleco de cuadros damero con cuello camisero, mangas sisa, bolsillos delanteros y cierre de botones.
['https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/04470305300', 'https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/04470522301', 'https://itxrest.inditex.com/LOMOServiciosRESTCommerce-ws/common/1/stock/campaign/V2021/product/part-number/04470451300']
{'datatype': 'stock', 'physicalStoreId': 11816, 'sizeStocks': [{'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102}, {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103}, {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}
{'datatype': 'sizeStock', 'sizeId': 

,Product_name,url,Image,Price,Quantity,Size,Tienda
0,"[Sobrecamisa oversize cuadros amarillos, Sobre...",[https://www.pullandbear.com/es/mujer-c1010141...,[https://static.pullandbear.net/2/photos/2021/...,"[28,79 € , 28,79 € , 28,79 € ]","Lo siento, no tenemos existencias!☹️",S,"[11816, 11816, 11816]"
0,"[Sobrecamisa oversize cuadros amarillos, Sobre...",[https://www.pullandbear.com/es/mujer-c1010141...,[https://static.pullandbear.net/2/photos/2021/...,"[28,79 € , 28,79 € , 28,79 € ]","Lo siento, no tenemos existencias!☹️",M,"[11816, 11816, 11816]"
0,"[Sobrecamisa oversize cuadros amarillos, Sobre...",[https://www.pullandbear.com/es/mujer-c1010141...,[https://static.pullandbear.net/2/photos/2021/...,"[28,79 € , 28,79 € , 28,79 € ]","Lo siento, no tenemos existencias!☹️",L,"[11816, 11816, 11816]"
1,"[Camisa tie-dye mancha amarilla, Camisa tie-dy...",[https://www.pullandbear.com/es/hombre-c101014...,[https://static.pullandbear.net/2/photos/2021/...,"[19,99 € , 19,99 € , 19,99 € , 19,99 € , 19,99...",El producto está en tienda!🙂,XS,"[11816, 11816, 11816, 11816, 11816]"
1,"[Camisa tie-dye mancha amarilla, Camisa tie-dy...",[https://www.pullandbear.com/es/hombre-c101014...,[https://static.pullandbear.net/2/photos/2021/...,"[19,99 € , 19,99 € , 19,99 € , 19,99 € , 19,99...",Últimas existencias 👀,XS,"[11816, 11816, 11816, 11816, 11816]"
1,"[Camisa tie-dye mancha amarilla, Camisa tie-dy...",[https://www.pullandbear.com/es/hombre-c101014...,[https://static.pullandbear.net/2/photos/2021/...,"[19,99 € , 19,99 € , 19,99 € , 19,99 € , 19,99...",El producto está en tienda!🙂,S,"[11816, 11816, 11816, 11816, 11816]"
1,"[Camisa tie-dye mancha amarilla, Camisa tie-dy...",[https://www.pullandbear.com/es/hombre-c101014...,[https://static.pullandbear.net/2/photos/2021/...,"[19,99 € , 19,99 € , 19,99 € , 19,99 € , 19,99...",Últimas existencias 👀,S,"[11816, 11816, 11816, 11816, 11816]"
1,"[Camisa tie-dye mancha amarilla, Camisa tie-dy...",[https://www.pullandbear.com/es/hombre-c101014...,[https://static.pullandbear.net/2/photos/2021/...,"[19,99 € , 19,99 € , 19,99 € , 19,99 € , 19,99...",El producto está en tienda!🙂,M,"[11816, 11816, 11816, 11816, 11816]"
1,"[Camisa tie-dye mancha amarilla, Camisa tie-dy...",[https://www.pullandbear.com/es/hombre-c101014...,[https://static.pullandbear.net/2/photos/2021/...,"[19,99 € , 19,99 € , 19,99 € , 19,99 € , 19,99...",Últimas existencias 👀,M,"[11816, 11816, 11816, 11816, 11816]"
1,"[Camisa tie-dye mancha amarilla, Camisa tie-dy...",[https://www.pullandbear.com/es/hombre-c101014...,[https://static.pullandbear.net/2/photos/2021/...,"[19,99 € , 19,99 € , 19,99 € , 19,99 € , 19,99...",El producto está en tienda!🙂,L,"[11816, 11816, 11816, 11816, 11816]"


In [2]:
def products_final_pull(search_item, lat, long):
    
    prod = get_stock_pull(search_item,lat,long)
    
    
    for a,product in enumerate(prod):
        
        product_id = product["stock"]["stocks"]["physicalStoreId"]
        
        name_ids = get_id_pull(product_name)
     
        products[a]["id_prod"] = name_ids
        
    for produc in prod:
        if type(produc["stock"]) == dict:
            pass
        else:
            produc["stock"] = ast.literal_eval(produc["stock"])
    sleep(2)
    
    
    for product_ in range(0, len(prod)):
    
        diccionario = {}
        diccionario["Pincha aquí para comprar el producto:"] = product_["url"]
        diccionario["Pincha aquí para ver como queda puesto!:"] = product_["image"]
        diccionario["Tipo de producto:"]= product_["product_name"]
        diccionario["Precio:"]= product_["price"]
    
        for stock in product_["stock"]["stocks"]:
       
            size_stk = []
        
            for sizeStock in stock["sizeStocks"]: 
            
                size_stock = {}
            
          
                size_ = {101:"XS", 102:"S", 103:"M", 104:"L", 105:"XL", 106:"XXL",125:"S-M",131:"L-XL"}
                if sizeStock["size"] in size_:
                    sizeStock["size"] = size_[sizeStock["size"]]
               
                    size_stock["Talla"] = sizeStock["size"]
                else:
                    size_stock["Talla"] = sizeStock["size"]
                
           
                if sizeStock["quantity"] <= 0:
                   size_stock["Stock"] = "Lo siento, no tenemos existencias!☹️"
                elif sizeStock["quantity"] == 1:
                    size_stock["Stock"] = "Últimas existencias!!"
                
                elif sizeStock["quantity"] >= 2:
                    size_stock["Stock"] = "El producto está en tienda!🙂"
                
                elif sizeStock["quantity"] >= 4:
                    size_stock["Stock"] = "Suficientes existencias!!😁"
                
                if size_stock["Stock"] == "No tenemos existencias! 🥲":
                    pass
                else: 
                    size_stk.append(size_stock)
            
        
        
            if len(size_stk) > 0:
                print("id tienda:",stock["physicalStoreId"])
                df = pd.DataFrame(size_stk)
                print(df)
                 
  

In [3]:
def products_final_pull(search_item, lat, long):
    
    prod = get_stock_pull(search_item,lat,long)
    
    for produc in prod:
        if type(produc["stock"]) == dict:
            pass
        else:
            produc["stock"] = ast.literal_eval(produc["stock"])
    sleep(2)
    
    
    for product_ in range(0, len(prod)):
    
        diccionario = {}
        diccionario["Pincha aquí para comprar el producto:"] = product_["url"]
        diccionario["Pincha aquí para ver como queda puesto!:"] = product_["image"]
        diccionario["Tipo de producto:"]= product_["product_name"]
        diccionario["Precio:"]= product_["price"]
    
        for stock in product_["stock"]["stocks"]:
       
            size_stk = []
        
            for sizeStock in stock["sizeStocks"]: 
            
                size_stock = {}
            
          
                size_ = {101:"XS", 102:"S", 103:"M", 104:"L", 105:"XL", 106:"XXL",125:"S-M",131:"L-XL"}
                if sizeStock["size"] in size_:
                    sizeStock["size"] = size_[sizeStock["size"]]
               
                    size_stock["Talla"] = sizeStock["size"]
                else:
                    size_stock["Talla"] = sizeStock["size"]
                
           
                if sizeStock["quantity"] <= 0:
                   size_stock["Stock"] = "Lo siento, no tenemos existencias!☹️"
                elif sizeStock["quantity"] == 1:
                    size_stock["Stock"] = "Últimas existencias!!"
                
                elif sizeStock["quantity"] >= 2:
                    size_stock["Stock"] = "El producto está en tienda!🙂"
                
                elif sizeStock["quantity"] >= 4:
                    size_stock["Stock"] = "Suficientes existencias!!😁"
                
                if size_stock["Stock"] == "No tenemos existencias! 🥲":
                    pass
                else: 
                    size_stk.append(size_stock)
            
        
        
            if len(size_stk) > 0:
                print("id tienda:",stock["physicalStoreId"])
                df = pd.DataFrame(size_stk)
                print(df)
                 

NameError: name 'get_stock_pull' is not defined

In [ ]:
chaqueta_morada = [{'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502662874.html?colorId=612',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/4582/313/612/4582313612_2_1_2.jpg?t=1614246566032',
  'product_name': 'Chaqueta canalé malva',
  'price': '17,99 € ',
  'id_prod': ['4582313612'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 11816,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5180,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 6601,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 12194,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5949,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5121,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 10510,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5259,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5390,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5204,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5010,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 921,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 2,
       'quantity': 0,
       'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}},
 {'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502548832.html?colorId=653',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/4582/306/653/4582306653_2_1_2.jpg?t=1611667786048',
  'product_name': 'Chaqueta pelito botones',
  'price': '15,99 € ',
  'id_prod': ['4582306512'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 11816,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5180,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 6601,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 12194,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5949,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 1, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 1, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5121,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 10510,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5259,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5390,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 1, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5204,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5010,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5346,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 921,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}},
 {'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502660734.html?colorId=629',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/4574/303/629/4574303629_2_1_2.jpg?t=1613464621559',
  'product_name': 'Chaleco ochos corto',
  'price': '15,99 € ',
  'id_prod': ['4574303300'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 11816,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5180,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 6601,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 12194,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5949,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5121,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 10510,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5259,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5390,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5204,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 1, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5010,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5346,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 921,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}},
 {'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502539262.html?colorId=629',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/9553/314/629/9553314629_2_1_2.jpg?t=1610706028338',
  'product_name': 'Jersey básico punto manga amplia',
  'price': '12,99 € ',
  'id_prod': ['9553314406'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 11816,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 1, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5180,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 6601,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 12194,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5949,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5121,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 10510,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5259,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5390,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5204,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5010,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5346,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 921,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}},
 {'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502548360.html?colorId=659',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/4556/309/659/4556309659_2_1_2.jpg?t=1607015320046',
  'product_name': 'Jersey cropped aberturas laterales',
  'price': '10,79 € ',
  'id_prod': ['4556309659'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 11816,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5180,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 6601,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 12194,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5949,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5121,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 10510,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5259,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5390,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5204,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5010,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5346,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 921,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}}]


In [ ]:
chaleco_morado  =  [{'url': 'https://www.pullandbear.com/es/mujer-c1010141503p502660734.html?colorId=629',
  'image': 'https://static.pullandbear.net/2/photos/2021/V/0/1/p/4574/303/629/4574303629_2_1_2.jpg?t=1613464621559',
  'product_name': 'Chaleco ochos corto',
  'price': '15,99 € ',
  'id_prod': ['4574303300'],
  'stock': {'stocks': [{'datatype': 'stock',
     'physicalStoreId': 11816,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5180,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 6601,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 12194,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5949,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 13636,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5239,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5121,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 10510,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 0, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5259,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5390,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5204,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104},
      {'datatype': 'sizeStock', 'sizeId': 5, 'quantity': 1, 'size': 105}]},
    {'datatype': 'stock',
     'physicalStoreId': 5010,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 5346,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]},
    {'datatype': 'stock',
     'physicalStoreId': 921,
     'sizeStocks': [{'datatype': 'sizeStock',
       'sizeId': 1,
       'quantity': 0,
       'size': 101},
      {'datatype': 'sizeStock', 'sizeId': 2, 'quantity': 0, 'size': 102},
      {'datatype': 'sizeStock', 'sizeId': 3, 'quantity': 0, 'size': 103},
      {'datatype': 'sizeStock', 'sizeId': 4, 'quantity': 0, 'size': 104}]}]}}]
